In [1]:
# Install the altair library for plotting
!pip install -U altair

# Clustering Crypto

In [35]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [3]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [4]:
# Create a DataFrame 
# Get crypto data using call to CryptoCompare API endpoint
crypto_response_data = requests.get(url)

# Convert response from api call to json format
crypto_response_data_json = crypto_response_data.json()

# Reach into json response content and select the Data then Create DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_data = pd.DataFrame(crypto_response_data_json["Data"])

# Transponse Date Frame and inspect
crypto_df = crypto_data.T
crypto_df.head()


,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,1.662354,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame


### Data Preprocessing

In [6]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
#crypto_df = crypto_df.filter(['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'])
#crypto_df.head()

#Note: The dataframe does not have a column = TotalCoinSupply, So I am using athe column = MaxSupply
crypto_df = crypto_df.filter(['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply'])
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
433,433 Token,N/A,False,N/A,NaN,NaN
...,...,...,...,...,...,...
GRBE,Green Beli,N/A,False,N/A,1000000000,-1
ATK,Attack Wagon,N/A,True,N/A,1000000000,-1
AFK,AFKDAO,N/A,True,N/A,0,-1
MENGO,Flamengo Fan Token,N/A,False,N/A,30000000,-1


In [7]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [8]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [9]:
# Remove the "IsTrading" column
crypto_df.drop(columns = 'IsTrading', inplace=True)

In [10]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)

In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]         #Select only cryptos that have mined coins > 0

In [12]:
# Drop rows where there are 'N/A' text values
na_columns = crypto_df.columns[crypto_df.isin(['N/A']).any()]     #Identify all columns that have N/A

for i in na_columns:
    crypto_df = crypto_df.loc[crypto_df[i] != 'N/A']              #Drop rows that have n/a in the selected column

In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df['CoinName'])
coins_name.set_index(crypto_df.index)
coins_name.head(10)

,CoinName
42,42 Coin
NSR,NuShares
TRI,Triangles Coin


In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns="CoinName", inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,41.999952,42
NSR,PoS,PoS,6178318685.8373,0
TRI,X13,PoW/PoS,191626.803418,0
CMTC,Scrypt,PoW,872830,0
CHAT,Scrypt,PoW/PoS,1000000000,-1


In [29]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])

In [30]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.08660377 -0.09087225 -0.08481889 -0.08481889 -0.08481889 -0.12038585
  -0.08481889 -0.08481889 -0.12038585 -0.12038585 -0.14797909 -0.08481889
  -0.08481889 -0.08481889 -0.24618298 -0.12038585 -0.08481889 -0.08481889
  -0.08481889 -0.29201253 -0.08481889 -0.08481889 -0.24618298 -0.08481889
  -0.08481889 -0.12038585 -0.08481889 -0.08481889 -0.08481889 -0.08481889
  -0.08481889 -0.08481889 -0.14797909 -0.08481889 -0.08481889 -0.12038585
  -0.19245009 -0.08481889 -0.08481889 -0.14797909 -0.12038585 -0.29201253
  -0.12038585 -0.08481889 -0.08481889 -0.08481889  2.19848433 -0.08481889
  -0.08481889 -0.08481889 -0.08481889 -0.08481889 -0.21160368 -0.08481889
  -0.19245009 -0.12038585 -0.08481889 -0.08481889 -0.08481889 -0.08481889
  -0.08481889 -0.26211122 -0.08481889 -0.08481889 -0.12038585 -0.12038585
  -0.08481889 -0.31994094 -0.08481889 -0.08481889 -0.08481889 -0.94440028
   2.         -0.08481889 -0.08481889 -0.08481889 -0.08481889 -0.08481889
  -0.08481889 -0.08481889 -0.08481889 

### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [18]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

pcs_df.head(10)

,PC 1,PC 2,PC 3
42,0.222851,-1.326062,-1.343659
NSR,0.696263,-1.169298,-0.312068
TRI,0.654794,-1.974567,-1.626911
CMTC,-0.853522,0.447471,-0.364627
CHAT,0.222856,-1.326058,-1.343659
PURA,-0.555574,0.116552,-0.338081
ADK,-0.929555,0.889838,0.289141
DAPS,0.861759,-1.951925,6.340259
VEIL,0.619081,-1.980336,-1.713641
RVC,-1.219943,1.247934,0.226503


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [36]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Plot the Elbow Curve using Altair Scatter Plot
alt.Chart(df_elbow).mark_circle(size=60).encode(
    x="k", 
    y="inertia", 
    color=alt.Color('inertia', scale=alt.Scale(scheme='viridis')),
    tooltip=['inertia', 'k']
)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


alt.Chart(...)

Running K-Means with `k=5`

In [38]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "MaxSupply": crypto_df.MaxSupply,                         #<-- TotalCoinSupply does not exist beacuse of API change, so using MaxSupply
    "PC 1": pcs_df['PC 1'],
    "PC 2": pcs_df['PC 2'],
    "PC 3": pcs_df['PC 3'],
    "CoinName": coins_name.CoinName,
    "Class": model.labels_,
    },
    index=crypto_df.index
)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42.0,0.222851,-1.326062,-1.343659,42 Coin,0
NSR,PoS,PoS,6.178319e+09,0.0,0.696263,-1.169298,-0.312068,NuShares,0
TRI,X13,PoW/PoS,1.916268e+05,0.0,0.654794,-1.974567,-1.626911,Triangles Coin,0
CMTC,Scrypt,PoW,8.728300e+05,0.0,-0.853522,0.447471,-0.364627,CometCoin,1
CHAT,Scrypt,PoW/PoS,1.000000e+09,-1.0,0.222856,-1.326058,-1.343659,OpenChat,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [39]:
# Scale data to create the scatter plot
clustered_scaled_df = pd.DataFrame(StandardScaler().fit_transform(clustered_df[["TotalCoinsMined", "MaxSupply"]]), index=clustered_df.index)
clustered_scaled_df.head(15)

,0,1
42,-0.086604,-0.090872
NSR,-0.086530,-0.090872
TRI,-0.086604,-0.090872
CMTC,-0.086604,-0.090872
CHAT,-0.086592,-0.090872
PURA,-0.086602,-0.090872
ADK,-0.086603,-0.090872
DAPS,-0.085856,-0.051316
VEIL,-0.086602,-0.090703
RVC,-0.086478,-0.079005


In [40]:
df = pd.DataFrame({
    "Algorithm": clustered_df.Algorithm,
    "ProofType": clustered_df.ProofType,
    "TotalCoinsMined": clustered_scaled_df[0],
    "MaxSupply": clustered_scaled_df[1],
    "PC 1": clustered_df['PC 1'],
    "PC 2": clustered_df['PC 2'],
    "PC 3": clustered_df['PC 3'],
    "CoinName": clustered_df.CoinName,
    "Class": clustered_df.Class,
    },
    index=crypto_df.index
)
df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,-0.086604,-0.090872,0.222851,-1.326062,-1.343659,42 Coin,0
NSR,PoS,PoS,-0.086530,-0.090872,0.696263,-1.169298,-0.312068,NuShares,0
TRI,X13,PoW/PoS,-0.086604,-0.090872,0.654794,-1.974567,-1.626911,Triangles Coin,0
CMTC,Scrypt,PoW,-0.086604,-0.090872,-0.853522,0.447471,-0.364627,CometCoin,1
CHAT,Scrypt,PoW/PoS,-0.086592,-0.090872,0.222856,-1.326058,-1.343659,OpenChat,0


In [41]:
# Use the altair scatter plot to visualize the clusters, x="PC 1" and y="PC 2" for the axes
alt.Chart(df).mark_circle(size=60).encode(
    x="PC 1", 
    y="PC 2", 
    color=alt.Color('Class', scale=alt.Scale(scheme='viridis')),
    # Add the required tool tips "CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "MaxSupply"]
)


alt.Chart(...)

In [42]:
#Use the altair scatter plot to visualize the tradable cryptocurrencies, x="TotalCoinsMined" and y="MaxSupply" for the axes
alt.Chart(df).mark_circle(size=60).encode(
    x="TotalCoinsMined",
    y="MaxSupply",
    color=alt.Color('Class', scale=alt.Scale(scheme='viridis')),
    # Add the required tool tips "CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "MaxSupply"]
)

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [45]:
# Print the total number of tradable cryptocurrencies using the display() command
display(clustered_df[['CoinName','Algorithm', 'ProofType', 'MaxSupply', 'TotalCoinsMined', 'Class']])

,CoinName,Algorithm,ProofType,MaxSupply,TotalCoinsMined,Class
42,42 Coin,Scrypt,PoW/PoS,4.200000e+01,4.199995e+01,0
AEON,AEON,CryptoNight-Lite,PoW,-1.000000e+00,1.801977e+07,1
ARK,ARK,DPoS,DPoS,-1.000000e+00,1.653636e+08,3
ACT,Achain,DPoS,DPoS,0.000000e+00,1.000000e+09,3
AAC,Acute Angle Cloud,ECC 256K1,DPOS,-1.000000e+00,1.000000e+09,2
...,...,...,...,...,...,...
XDNA,XDNA,HEX,PoW/PoS,0.000000e+00,8.830765e+06,0
ZEC,ZCash,Equihash,PoW,2.100000e+07,1.247692e+07,1
ZANO,Zano,ProgPowZ,PoW/PoS,-1.000000e+00,1.320731e+07,0
ZEL,Zelcash,Equihash,PoW/PoS,2.100000e+08,4.400000e+08,0
